<a href="https://colab.research.google.com/github/pratamastan/bangkit-capstone0143/blob/main/NLP_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Fetching and train-test split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
# import pathlib
# from os import getcwd

Training data is taken from Quora Questions Pairs dataset (training set only) https://www.kaggle.com/c/quora-question-pairs/overview translated to Indonesian with Google Translate

In [ ]:
data1=pd.read_csv("/content/drive/MyDrive/Quora question pair dataset/train complete indo.csv")

In [ ]:
data1=data1.dropna()

In [ ]:
data1.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,Apa panduan langkah demi langkah untuk berinve...,Apa panduan langkah demi langkah untuk berinve...,0
1,1,3,4,Bagaimana kisah Berlian Kohinoor (Koh-i-Noor)?,Apa yang akan terjadi jika pemerintah India me...,0
2,2,5,6,Bagaimana cara meningkatkan kecepatan koneksi ...,Bagaimana kecepatan Internet dapat ditingkatka...,0
3,3,7,8,Mengapa saya secara mental sangat kesepian? Ba...,Temukan sisanya ketika [math] 23 ^ {24} [/ mat...,0
4,4,9,10,Manakah yang larut dalam air seperti gula pasi...,Ikan apa yang akan bertahan hidup di air asin?,0


In [ ]:
data1.question1.str.len().max()

717

In [ ]:
data1.question2.str.len().max()

1395

Train-test split 80% 20%

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data1[["question1", "question2"]], data1["is_duplicate"], test_size=0.2, random_state=42)

Transform the dataframe to NumPy array as TensorFlow takes NumPy arrays

In [ ]:
xt=X_train.to_numpy()
yt=y_train.to_numpy()

xte=X_test.to_numpy()
yte=y_test.to_numpy()

Q1_train = X_train["question1"].to_numpy()
Q2_train = X_train["question2"].to_numpy()
Q1_test = X_test["question1"].to_numpy()
Q2_test = X_test["question2"].to_numpy()

In [ ]:
Q1_train

array(['Apa sajakah cerita pendeknya?',
       'Apa jawaban terbaik untuk pertanyaan HR, "Anda berasal dari latar belakang ECE, jadi mengapa Anda ingin bergabung dengan perusahaan IT"?',
       'Bagaimana cara membulatkan 1,44 ke persen terdekat?', ...,
       'Siapa yang saat ini memenangkan pemilihan presiden?',
       'Apa sajakah film Telugu / Tamil / Hindi yang pemeran utamanya menjadi mafia atau tidak?',
       'Berapa peringkat yang dipotong untuk perguruan tinggi Kedokteran Rohtak dalam kuota negara bagian?'],
      dtype=object)

In [ ]:
data1["question1"][0]

'Apa panduan langkah demi langkah untuk berinvestasi di pasar saham di India?'

In [ ]:
data1[["question1","question2"]]

,question1,question2
0,Apa panduan langkah demi langkah untuk berinve...,Apa panduan langkah demi langkah untuk berinve...
1,Bagaimana kisah Berlian Kohinoor (Koh-i-Noor)?,Apa yang akan terjadi jika pemerintah India me...
2,Bagaimana cara meningkatkan kecepatan koneksi ...,Bagaimana kecepatan Internet dapat ditingkatka...
3,Mengapa saya secara mental sangat kesepian? Ba...,Temukan sisanya ketika [math] 23 ^ {24} [/ mat...
4,Manakah yang larut dalam air seperti gula pasi...,Ikan apa yang akan bertahan hidup di air asin?
...,...,...
404285,Ada berapa kata kunci dalam bahasa pemrograman...,Ada berapa kata kunci dalam Bahasa Pemrograman...
404286,Apakah Anda percaya ada kehidupan setelah kema...,Benarkah ada kehidupan setelah kematian?
404287,Apa satu koin?,Koin apa ini?
404288,Berapa perkiraan biaya hidup tahunan selama be...,Saya mengalami sedikit masalah rambut rontok t...


# Transfer Learning for Indonesian language embedding
### Embedding to vector with 128 dimension

In [ ]:
#Model runtime will increase if tranable=True to about 900-1000s per epoch
embed = hub.KerasLayer("https://tfhub.dev/google/nnlm-id-dim128-with-normalization/2",input_shape=[], dtype=tf.string)
#embed.trainable=True

In [ ]:
xt1=embed(Q1_train)
xt2=embed(Q2_train)
xte1=embed(Q1_test)
xte2=embed(Q2_test)

In [ ]:
xt1

<tf.Tensor: shape=(323430, 128), dtype=float32, numpy=
array([[ 0.03792422, -0.08203722, -0.03952134, ..., -0.03751129,
        -0.13026127, -0.21294448],
       [ 0.16049062,  0.05221158, -0.03756306, ..., -0.06843793,
        -0.16083518,  0.00526717],
       [ 0.00541904,  0.03682816, -0.10739181, ..., -0.05703003,
        -0.12414219, -0.05216019],
       ...,
       [ 0.17406467,  0.06946962, -0.02178098, ..., -0.05727712,
        -0.17851247, -0.00691927],
       [ 0.14288372, -0.16400667,  0.03347619, ...,  0.04388543,
        -0.12863201, -0.02391369],
       [-0.02106932, -0.22670205, -0.11702876, ..., -0.13742013,
        -0.11424292,  0.08969257]], dtype=float32)>

# Base Models without embedding integrated

In [ ]:
##Will run 1 epoch about 2 minutes
model=tf.keras.Sequential([tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                           tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(1, activation="sigmoid")
])
#model.build((1,2,128))

In [ ]:
#fit the model first and then run this cell, if not, this cell won't run properly
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 2, 128)            98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 200)               13000     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 200)               800       
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(np.stack((xt1, xt2), axis=1),
                    yt,
                    epochs=5,
                    validation_data=(np.stack((xte1, xte2), axis=1), yte),
                    verbose=2)

Epoch 1/5
10108/10108 - 127s - loss: 0.5713 - accuracy: 0.7028 - val_loss: 0.5203 - val_accuracy: 0.7332
Epoch 2/5
10108/10108 - 89s - loss: 0.4990 - accuracy: 0.7478 - val_loss: 0.4760 - val_accuracy: 0.7597
Epoch 3/5
10108/10108 - 87s - loss: 0.4668 - accuracy: 0.7676 - val_loss: 0.4600 - val_accuracy: 0.7681
Epoch 4/5
10108/10108 - 86s - loss: 0.4468 - accuracy: 0.7809 - val_loss: 0.4450 - val_accuracy: 0.7805
Epoch 5/5
10108/10108 - 86s - loss: 0.4299 - accuracy: 0.7923 - val_loss: 0.4362 - val_accuracy: 0.7875


In [ ]:
xt[0].shape

(2,)

In [ ]:
## 1 epoch is about 150-200s, added GRU Layer between the stacked Bidirectional LSTM layer
model3=tf.keras.Sequential([tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                            tf.keras.layers.GRU(64, return_sequences=True),
                            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                            tf.keras.layers.Dense(256,activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(256,activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(256,activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(256,activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(256,activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
#fit the model first and then run this cell, if not, this cell won't run properly
model3.summary()

In [ ]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history3 = model3.fit(np.stack((xt1, xt2), axis=1),
                    yt,
                    epochs=5,
                    validation_data=(np.stack((xte1, xte2), axis=1), yte),
                    verbose=2)

Epoch 1/5
10108/10108 - 172s - loss: 0.5752 - accuracy: 0.7000 - val_loss: 0.5065 - val_accuracy: 0.7385
Epoch 2/5
10108/10108 - 166s - loss: 0.4993 - accuracy: 0.7466 - val_loss: 0.4812 - val_accuracy: 0.7582
Epoch 3/5
10108/10108 - 169s - loss: 0.4704 - accuracy: 0.7663 - val_loss: 0.4679 - val_accuracy: 0.7670
Epoch 4/5
10108/10108 - 171s - loss: 0.4508 - accuracy: 0.7796 - val_loss: 0.4520 - val_accuracy: 0.7793
Epoch 5/5
10108/10108 - 166s - loss: 0.4343 - accuracy: 0.7901 - val_loss: 0.4396 - val_accuracy: 0.7847


In [ ]:
a=embed(["saya memukul kursi"])
a2=embed(["kursi dipukul oleh saya"])

In [ ]:
prediction = model.predict(np.stack((a,a2),axis=1))
print(prediction)

[[0.6698727]]


In [ ]:
prediction = model3.predict(np.stack((a,a2),axis=1))
print(prediction)

[[0.6323676]]


# Models with two inputs and embedding integrated into the models

In [ ]:
## Model without Bidirectional LSTM model accuracy is only 66-72%, take 2 inputs, will run 1 epoch about 90s 
# ques1=tf.keras.Input(shape=(), dtype=tf.string)
# ques2=tf.keras.Input(shape=(), dtype=tf.string)

# qu1=embed(ques1)
# qu2=embed(ques2)
# attention=tf.keras.layers.Attention()([qu1,qu2])
# #attention=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(attention)
# #attention=tf.keras.layers.Attention()(attention)
# attention=tf.keras.layers.Flatten()(attention)
# attention=tf.keras.layers.Dense(300, activation="relu")(attention)
# merged = tf.keras.layers.Dense(200,activation='relu')(attention)
# merged = tf.keras.layers.Dropout(0.2)(merged)
# merged = tf.keras.layers.BatchNormalization()(merged)
# merged = tf.keras.layers.Dense(200,activation='relu')(merged)
# merged = tf.keras.layers.Dropout(0.2)(merged)
# merged = tf.keras.layers.BatchNormalization()(merged)
# merged = tf.keras.layers.Dense(200,activation='relu')(merged)
# merged = tf.keras.layers.Dropout(0.2)(merged)
# merged = tf.keras.layers.BatchNormalization()(merged)
# merged = tf.keras.layers.Dense(200,activation='relu')(merged)
# merged = tf.keras.layers.Dropout(0.2)(merged)
# merged = tf.keras.layers.BatchNormalization()(merged)

# is_duplicate = tf.keras.layers.Dense(1, activation="sigmoid")(merged)
# model2 = tf.keras.models.Model(inputs=[ques1,ques2], outputs=is_duplicate)
# model2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 128)          112461824   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
attention (Attention)           (None, 128)          0           keras_layer[2][0]            

In [ ]:
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# history2 = model2.fit([Q1_train, Q2_train],
#                     yt,
#                     epochs=5,
#                     validation_data=([Q1_test,Q2_test], yte),
#                     verbose=2)

Epoch 1/5
10108/10108 - 87s - loss: 0.6506 - accuracy: 0.6347 - val_loss: 1.0374 - val_accuracy: 0.5443
Epoch 2/5
10108/10108 - 87s - loss: 0.6365 - accuracy: 0.6497 - val_loss: 0.6807 - val_accuracy: 0.6189
Epoch 3/5
10108/10108 - 89s - loss: 0.6330 - accuracy: 0.6540 - val_loss: 0.7115 - val_accuracy: 0.6095
Epoch 4/5
10108/10108 - 86s - loss: 0.6312 - accuracy: 0.6559 - val_loss: 0.8011 - val_accuracy: 0.5844
Epoch 5/5
10108/10108 - 89s - loss: 0.6305 - accuracy: 0.6563 - val_loss: 0.6913 - val_accuracy: 0.6308


In [ ]:
#Model with Bidirectional LSTM, Attention layer, take 2 inputs, will run about 150s for 1 epoch
ques1=tf.keras.Input(shape=(), dtype=tf.string)
ques2=tf.keras.Input(shape=(), dtype=tf.string)

qu1=embed(ques1)
qu2=embed(ques2)

q1=tf.expand_dims(qu1, axis=1)
q2=tf.expand_dims(qu2, axis=1)
q1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(q1)
q2=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(q2)

attention=tf.keras.layers.Attention()([q1,q2])
#attention=tf.keras.layers.Flatten()(attention)
attention=tf.keras.layers.Dense(300, activation="relu")(attention)
#attention=tf.keras.layers.Reshape((2,128))(attention)

#merged = tf.keras.layers.Add([q1,attention])
#merged = tf.keras.layers.Flatten()(attention)
merged = tf.keras.layers.Dense(200,activation='relu')(attention)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)
merged = tf.keras.layers.Dense(200,activation='relu')(merged)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)
merged = tf.keras.layers.Dense(200,activation='relu')(merged)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)
merged = tf.keras.layers.Dense(200,activation='relu')(merged)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)

is_duplicate = tf.keras.layers.Dense(1, activation="sigmoid")(merged)
model1 = tf.keras.models.Model(inputs=[ques1,ques2], outputs=is_duplicate)
model1.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 128)          112461824   input_21[0][0]                   
                                                                 input_22[0][0]                   
__________________________________________________________________________________________________
tf.expand_dims_6 (TFOpLambda)   (None, 1, 128)       0           keras_layer[10][0]         

In [ ]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history1 = model1.fit([Q1_train, Q2_train],
                    yt,
                    epochs=5,
                    validation_data=([Q1_test,Q2_test], yte),
                    verbose=2)

Epoch 1/5
10108/10108 - 158s - loss: 0.6065 - accuracy: 0.6718 - val_loss: 0.5756 - val_accuracy: 0.6970
Epoch 2/5
10108/10108 - 150s - loss: 0.5720 - accuracy: 0.7036 - val_loss: 0.5628 - val_accuracy: 0.7107
Epoch 3/5
10108/10108 - 150s - loss: 0.5572 - accuracy: 0.7153 - val_loss: 0.5507 - val_accuracy: 0.7214
Epoch 4/5
10108/10108 - 152s - loss: 0.5445 - accuracy: 0.7252 - val_loss: 0.5390 - val_accuracy: 0.7288
Epoch 5/5
10108/10108 - 149s - loss: 0.5343 - accuracy: 0.7333 - val_loss: 0.5355 - val_accuracy: 0.7312


In [ ]:
prediction1 = model1.predict([np.asarray(["dia memainkan gitar"]),np.asarray(["gitar dimainkan oleh dia"])])
print(prediction1)

[[[0.3716218]]]


# Final Models
## Models with two inputs and embedding integrated

## Model with GRU layer

In [ ]:
# Model with Bidirectional LSTM, GRU, without Attention, will run for about 150-200s per epoch
qstn1=tf.keras.Input(shape=(), dtype=tf.string)
qstn2=tf.keras.Input(shape=(), dtype=tf.string)

qu1=embed(qstn1)
qu2=embed(qstn2)
concat=tf.keras.layers.concatenate([qu1,qu2])
concat=tf.keras.layers.Reshape((2,128))(concat)
concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(concat)
concat=tf.keras.layers.GRU(64, return_sequences=True)(concat)
concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
is_dupe=tf.keras.layers.Dense(1, activation="sigmoid")(concat)

model4 = tf.keras.models.Model(inputs=[qstn1,qstn2], outputs=is_dupe)
model4.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 128)          112461824   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 256)          0           keras_layer[0][0]            

In [ ]:
# a=embed(["abc"])
# b=embed(["bcd"])
# aa=tf.keras.layers.Reshape((1,128))(a)
# bb=tf.keras.layers.Reshape((1,128))(b)
# ab=tf.keras.layers.concatenate([a,b])
# tf.keras.layers.Reshape((2,128))(ab)

In [ ]:
tf.keras.utils.plot_model(model4, show_shapes=True)

In [ ]:
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history4 = model4.fit([Q1_train, Q2_train],
                    yt,
                    epochs=10,
                    validation_data=([Q1_test,Q2_test], yte),
                    verbose=2)

Epoch 1/10
10108/10108 - 253s - loss: 0.5762 - accuracy: 0.6988 - val_loss: 0.5281 - val_accuracy: 0.7285
Epoch 2/10
10108/10108 - 218s - loss: 0.5016 - accuracy: 0.7464 - val_loss: 0.4768 - val_accuracy: 0.7610
Epoch 3/10
10108/10108 - 210s - loss: 0.4701 - accuracy: 0.7662 - val_loss: 0.4647 - val_accuracy: 0.7710
Epoch 4/10
10108/10108 - 211s - loss: 0.4495 - accuracy: 0.7799 - val_loss: 0.4487 - val_accuracy: 0.7809
Epoch 5/10
10108/10108 - 211s - loss: 0.4339 - accuracy: 0.7906 - val_loss: 0.4371 - val_accuracy: 0.7867
Epoch 6/10
10108/10108 - 212s - loss: 0.4195 - accuracy: 0.7997 - val_loss: 0.4341 - val_accuracy: 0.7876
Epoch 7/10
10108/10108 - 212s - loss: 0.4073 - accuracy: 0.8071 - val_loss: 0.4389 - val_accuracy: 0.7868
Epoch 8/10
10108/10108 - 211s - loss: 0.3961 - accuracy: 0.8139 - val_loss: 0.4296 - val_accuracy: 0.7960
Epoch 9/10
10108/10108 - 215s - loss: 0.3858 - accuracy: 0.8207 - val_loss: 0.4220 - val_accuracy: 0.7987
Epoch 10/10
10108/10108 - 213s - loss: 0.3761 

In [ ]:
prediction4 = model4.predict([np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])])
print(prediction4)

[[0.90333223]]


In [ ]:
model4.predict([np.asarray(["saya yang mempunyai gitar yamaha itu"]),np.asarray(["gitar yamaha itu milik saya"])])

array([[0.84960437]], dtype=float32)

In [ ]:
model4.predict([np.asarray(["saya bermain di atas meja"]),np.asarray(["kursi diduduki oleh saya"])])

array([[0.05997301]], dtype=float32)

### Save the first model

In [ ]:
export_dir1 = 'saved_model/model_gru1'
tf.saved_model.save(model4, export_dir=export_dir1)

INFO:tensorflow:Assets written to: saved_model/model_gru1/assets


INFO:tensorflow:Assets written to: saved_model/model_gru1/assets


In [ ]:
new_model4 = tf.keras.models.load_model(export_dir1)
# nnew_model4 = tf.keras.models.load_model("/content/drive/MyDrive/saved_model/model_gru1")
new_model4.summary()
# nnew_model4.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 128)          112461824   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 256)          0           keras_layer[0][0]            

In [ ]:
new_model4.predict([np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])])
# nnew_model4.predict([np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])])

array([[0.90333223]], dtype=float32)

### TFLite conversion for the GRU model (does not work unfortunately)

In [ ]:
# optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
converter.allow_custom_ops = True
tflite_model = converter.convert()

ConverterError: ignored

## Model without GRU Layer

In [ ]:
# Model with Bidirectional LSTM, without Attention, will run for about 150-200s per epoch
qstn1=tf.keras.Input(shape=(), dtype=tf.string)
qstn2=tf.keras.Input(shape=(), dtype=tf.string)

qu1=embed(qstn1)
# qu1= hub.KerasLayer("https://tfhub.dev/google/nnlm-id-dim128-with-normalization/2",input_shape=[], dtype=tf.string, trainable=False)(qstn1)
qu2=embed(qstn2)
# qu2= hub.KerasLayer("https://tfhub.dev/google/nnlm-id-dim128-with-normalization/2",input_shape=[], dtype=tf.string, trainable=False)(qstn2)
concat=tf.keras.layers.concatenate([qu1,qu2])
concat=tf.keras.layers.Reshape((2,128))(concat)
concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(concat)
concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
is_dupe=tf.keras.layers.Dense(1, activation="sigmoid")(concat)

model5 = tf.keras.models.Model(inputs=[qstn1,qstn2], outputs=is_dupe)
model5.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      (None, 128)          112461824   input_3[0][0]                    
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      (None, 128)          112461824   input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
tf.keras.utils.plot_model(model5, show_shapes=True)

In [ ]:
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history5 = model5.fit([Q1_train, Q2_train],
                    yt,
                    epochs=10,
                    validation_data=([Q1_test,Q2_test], yte),
                    verbose=2)

Epoch 1/10
10108/10108 - 200s - loss: 0.5704 - accuracy: 0.7023 - val_loss: 0.5327 - val_accuracy: 0.7357
Epoch 2/10
10108/10108 - 192s - loss: 0.5002 - accuracy: 0.7468 - val_loss: 0.4786 - val_accuracy: 0.7594
Epoch 3/10
10108/10108 - 191s - loss: 0.4695 - accuracy: 0.7658 - val_loss: 0.4754 - val_accuracy: 0.7636
Epoch 4/10
10108/10108 - 191s - loss: 0.4497 - accuracy: 0.7800 - val_loss: 0.4516 - val_accuracy: 0.7807
Epoch 5/10
10108/10108 - 193s - loss: 0.4329 - accuracy: 0.7914 - val_loss: 0.4403 - val_accuracy: 0.7866
Epoch 6/10
10108/10108 - 193s - loss: 0.4198 - accuracy: 0.7993 - val_loss: 0.4339 - val_accuracy: 0.7907
Epoch 7/10
10108/10108 - 193s - loss: 0.4071 - accuracy: 0.8072 - val_loss: 0.4233 - val_accuracy: 0.7950
Epoch 8/10
10108/10108 - 193s - loss: 0.3963 - accuracy: 0.8144 - val_loss: 0.4256 - val_accuracy: 0.7918
Epoch 9/10
10108/10108 - 196s - loss: 0.3852 - accuracy: 0.8201 - val_loss: 0.4348 - val_accuracy: 0.7929
Epoch 10/10
10108/10108 - 194s - loss: 0.3747 

In [ ]:
model5.predict([np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])])

array([[0.674969]], dtype=float32)

In [ ]:
model5.predict([np.asarray(["saya yang mempunyai gitar yamaha itu"]),np.asarray(["gitar yamaha itu milik saya"])])

array([[0.76135254]], dtype=float32)

In [ ]:
model5.predict([np.asarray(["saya bermain di atas meja"]),np.asarray(["kursi diduduki oleh saya"])])

array([[0.03563059]], dtype=float32)

### Save the second model

In [ ]:
export_dir2 = 'saved_model/model_2'
tf.saved_model.save(model5, export_dir=export_dir2)

INFO:tensorflow:Assets written to: saved_model/model_2/assets


INFO:tensorflow:Assets written to: saved_model/model_2/assets


In [ ]:
new_model5 = tf.keras.models.load_model(export_dir2)
# nnew_model5 = tf.keras.models.load_model("/content/drive/MyDrive/saved_model/model_2")
new_model5.summary()
# nnew_model5.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      (None, 128)          112461824   input_3[0][0]                    
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      (None, 128)          112461824   input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
new_model5.predict([np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])])
# nnew_model5.predict([np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])])

array([[0.674969]], dtype=float32)

### TFLite conversion (also fails for this model unfortunately)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model5)

# Set the optimzations
optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
converter.allow_custom_ops = True
tflite_model = converter.convert()

Exception ignored in: <function CapturableResourceDeleter.__del__ at 0x7fcdca490830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/p

INFO:tensorflow:Assets written to: /tmp/tmpt002z6jz/assets


INFO:tensorflow:Assets written to: /tmp/tmpt002z6jz/assets


ConverterError: ignored

In [ ]:
#DO NOT RUN as tflite conversion fails
tflite_model_file = pathlib.Path('./model4.tflite')

In [ ]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = [np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])]
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

# Model with diff embedding

It is not working right now, I haven't been able to make it work due to mismatch in dimension.

In [ ]:
#Embedding for Indonesian language (vector with 128 dimension)

# !pip install --quiet tensorflow_text
# import tensorflow.compat.v1 as tf1
# import tensorflow_text as tf_text
# tf1.disable_eager_execution()

embed1 = hub.KerasLayer("https://tfhub.dev/google/wiki40b-lm-id/1", signature='word_embeddings', output_key="word_embeddings")

In [ ]:
embed1(np.array(["abc"]))

<tf.Tensor 'keras_layer_8/StatefulPartitionedCall:0' shape=(None, None, 768) dtype=float32>

In [ ]:
# embeddings = embed2([Q1_train[0]], signature="word_embeddings",as_dict=True)["word_embeddings"]

In [ ]:
# xt1=embed(Q1_train)
# xt2=embed(Q2_train)
# xte1=embed(Q1_test)
# xte2=embed(Q2_test)

In [ ]:
qstn1=tf.keras.Input(shape=(), dtype=tf.string)
#qstn2=tf.keras.Input(shape=(), dtype=tf.string)
qu1=embed1(qstn1)
#qu2=embed1(qstn2)
#concat=tf.keras.layers.concatenate([qu1,qu2])
#concat=tf.keras.layers.Reshape((2,128))(concat)
#concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(concat)
#concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(concat)

model7 = tf.keras.models.Model(inputs=[qstn1], outputs=qu1)
model7.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
keras_layer_1 (KerasLayer)   {'word_embeddings': (None 116768000 
Total params: 116,768,000
Trainable params: 0
Non-trainable params: 116,768,000
_________________________________________________________________


In [ ]:
model7.predict(np.array(["abc"]))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


array([[[ 1.15195453e-01,  6.62501037e-01,  3.49963069e-01,
         -5.74863292e-02, -1.46612573e+00,  3.22761416e-01,
          7.45852351e-01,  9.10114825e-01,  3.02208424e-01,
          4.80806649e-01, -3.55580121e-01, -3.32741976e+00,
          2.47957751e-01,  4.46250051e-01, -2.48991847e-01,
          4.71701145e-01, -3.88547033e-02,  1.03991735e+00,
          4.21689242e-01,  1.58651364e+00, -1.02430892e+00,
         -1.03150338e-01,  2.25313857e-01, -6.67892337e-01,
          4.70242053e-01, -4.83143240e-01, -1.05122522e-01,
          3.94368261e-01, -1.20789266e+00, -5.23485243e-01,
          7.10996389e-02,  9.82663870e-01,  2.00165272e-01,
          8.26833844e-01, -8.62367034e-01,  9.92930979e-02,
         -6.68423116e-01, -3.84718865e-01, -3.90402526e-01,
         -5.94246507e-01,  5.88665605e-02, -5.26681602e-01,
          6.62307918e-01,  1.38507760e+00, -3.99313979e-02,
         -1.15141466e-01,  1.24471821e-02, -6.85775459e-01,
          1.04845572e+00, -1.35175213e-0

In [ ]:
#hub_layer = hub.KerasLayer("https://tfhub.dev/google/wiki40b-lm-id/1",signature='word_embeddings' , signature_outputs_as_dict=True,input_shape=[], dtype=tf.string)
qstn1=tf.keras.Input(shape=(), dtype=tf.string)
qstn2=tf.keras.Input(shape=(), dtype=tf.string)
qu1=embed1(qstn1)
qu2=embed1(qstn2)
concat=tf.keras.layers.concatenate([qu1,qu2])
#concat=tf.keras.layers.Reshape((2,768))(concat)
concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(384, return_sequences=True))(concat)
concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128))(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
is_dupe=tf.keras.layers.Dense(1, activation="sigmoid")(concat)

model6 = tf.keras.models.Model(inputs=[qstn1,qstn2], outputs=is_dupe)
model6.summary()


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_38 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
input_39 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_8 (KerasLayer)      (None, None, 768)    116768000   input_38[0][0]                   
                                                                 input_39[0][0]                   
__________________________________________________________________________________________________
concatenate_8 (Concatenate)     (None, None, 1536)   0           keras_layer_8[16][0]       

In [ ]:
model6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history6 = model6.fit([Q1_train, Q2_train],
                    yt,
                    epochs=5,
                    validation_data=([Q1_test,Q2_test], yte),
                    verbose=2)

Train on 323430 samples, validate on 80858 samples
Epoch 1/5


InvalidArgumentError: ignored

# Save the models to Drive

In [ ]:
#save model to MyDrive
!cp -r /content/saved_model /content/drive/MyDrive/